In [ ]:
!pip install openai
!pip install datasets
!pip install git+https://github.com/google-research/bleurt.git
!pip install transformers
!pip install bert_score
!git clone https://github.com/google-research/bleurt.git
%cd bleurt
!pip install .
%cd /content/
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip .
!unzip BLEURT-20.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.1 MB/s eta 0:0

In [ ]:
!pip install openai
!pip install sentence_transformers
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=96938b614120e412b3fee98dc12737c79acf15b2a03397d40c4dceb43cb0943d
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import sklearn
from sklearn import metrics
import json
import openai
import time
import random
from datasets import load_metric, load
from bleurt import score as bleurt_score
from bert_score import score as bert_score

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
test_df = pd.read_csv('../Datasets/prompting_test.csv')
train_df = pd.read_csv('../Datasets/train-0.3.csv')
premise_train_df = train_df.dropna(subset = ["Premise"])

In [ ]:
import torch
import random
import numpy as np

torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)
random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Confirm that the GPU is detected
assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


In [ ]:
prathiksha_api_key = "REPLACE WITH YOUR KEY"
aditya_api_key = "REPLACE WITH YOUR KEY"
openai.api_key = aditya_api_key

In [ ]:
def generate_few_shot_prompt(neighbours, df, type_considered=False):
  prompt = ""
  for i in neighbours:
    task_des = ("Give literal translation for this " + df.iloc[i]["Type"] + "\n") if type_considered else ""
    prompt = prompt + task_des + "Figurative sentence: " + df.iloc[i]["Hypothesis"] + "\nLiteral translation: " + df.iloc[i].Premise + "\n\n"
  return prompt

In [ ]:
def fewshot_random(input, k, df, type_filtering = False):
  if type_filtering:
    df = premise_train_df[premise_train_df["Type"]==input.Type]
  else :
    df = premise_train_df
  neighbours_indices = random.sample(range(df.shape[0]),k)
  few_shot_prompt = generate_few_shot_prompt(neighbours_indices, df, type_filtering)
  task_des = ("For the given figurative sentences, provide a literal translation: \n\n") if not type_filtering else ""
  test_sample_prompt = ("Give literal translation for this " + get_pred(input.Hypothesis) + "\n") if type_filtering else ""
  test_case = "Figurative sentence: " + input.Hypothesis + "\nLiteral translation: "
  prompt = task_des + few_shot_prompt +test_sample_prompt + test_case
  #print(prompt, end = "")
  lit_output = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt, 
    max_tokens = 1500
  )
  #print(lit_output.choices[0].text.strip('\n') + '\n')
  #print("Actual - ", input.Premise)
  return lit_output.choices[0].text.strip('\n')

In [ ]:
df_fewshot_random = test_df.copy()
df_fewshot_random["Literal Translation"] = [""]*df_fewshot_random.shape[0]
for i in range(0,df_fewshot_random.shape[0]):
  input = df_fewshot_random.iloc[i]
  pred = fewshot_random(input,3,False)
  print(pred)
  df_fewshot_random.loc[i,"Literal Translation"] = pred

 I heard a knock on my door at 2 am in the morning, and was worried because I didn't know who it was.
 I was disappointed to find out that the person who I admired for their great vision is actually rude to people.
 I am annoyed when people blast their music during the week at 3am and disrupt my sleep.
 I am exasperated because I do not need the additional stress of a headache to make my day more difficult.


KeyboardInterrupt: ignored

In [ ]:
df_fewshot_random

,Unnamed: 0,Index,Hypothesis,Premise,Type,Explanation,Label,SarcasmSimilarity,Source,Literal Translation
0,331,50,I heard a knock on my door at 2 am in the morn...,I heard a knock on my door at 2 am in the morn...,Sarcasm,Unknown people knocking on someone's door at 2...,Contradiction,UNQ,FLUTE,I heard a knock on my door at 2 am in the mor...
1,233,1630,I was very proud of my sense of judgement when...,I was seriously doubting my sense of judgement...,Sarcasm,: Someone who is admired for their great visio...,Contradiction,UNQ,FLUTE,I was disappointed to discover that the perso...
2,66,2112,I adore it when people blast their music durin...,I despise it when people blast their music dur...,Sarcasm,People blasting their music during the wee hou...,Contradiction,UNQ,FLUTE,I am angry and frustrated when people blast t...
3,374,1833,I am pleased because the last thing I need is ...,I've been getting really bad headaches recently,Sarcasm,Having a headache makes someone's day worse as...,Contradiction,UNQ,FLUTE,I am frustrated because the last thing I need...
4,452,1057,I was extremely happy to find out that all my ...,I was extremely broken to find out that all my...,Sarcasm,Studying for an annual exam is often a waste o...,Contradiction,UNQ,FLUTE,I was not pleased to find out that all my stu...
...,...,...,...,...,...,...,...,...,...,...
395,332,3260,I know that woman too well to ever want to lay...,I know that woman too well to ever want to hur...,Idiom,To lay a finger on someone usually refers to h...,Entailment,UNQ,FLUTE,I know that woman too well to ever want to to...
396,527,7919,‘ That spoke volumes for the character as well...,‘ That provided substantial information for th...,Idiom,NaN,Entailment,NaN,IMPLI,‘That showed that there is a lot of knowledge...
397,569,6303,It 's all water under the bridge ; you do n't ...,It 's all in the past ; you do n't want to hea...,Idiom,NaN,Entailment,NaN,IMPLI,It is all in the past now; you do not want to...
398,222,6481,The road and railway from Mallaig go south han...,The road and railway from Mallaig go south in ...,Idiom,NaN,Entailment,NaN,IMPLI,The road and railway from Mallaig go south to...


In [ ]:
def calculate_bleurt(df):
  # BLEURT calculation
  scorer = bleurt_score.BleurtScorer('/content/BLEURT-20')
  bleurt_scores = scorer.score(references=df['Premise'], candidates=df['Literal Translation'])
  print(bleurt_scores)
  return bleurt_scores

def calculate_bertscore(df):
  # BERTScore calculation
  bertscore = load_metric('bertscore')
  bert_scores = bertscore.compute(predictions=df['Literal Translation'], references=df['Premise'], lang="en")
  print(bert_scores['f1'])
  return bert_scores['f1']

def calculate_littransscore(bert_scores, bleurt_scores):
  lit_trans_scores = []
  for ind in range(len(bleurt_scores)):
    lit_trans_scores.append((bert_scores[ind] + bleurt_scores[ind]) * 50.0)
  return lit_trans_scores

In [ ]:
def calculate_scores(test_df):
  bleurt_scores = calculate_bleurt(test_df)
  bert_scores = calculate_bertscore(test_df)
  
  print("Range of BLEURT: ")
  print(min(bleurt_scores))
  print(max(bleurt_scores), end='\n')

  print("Range of BERTScore: ")
  print(min(bert_scores))
  print(max(bert_scores), end='\n')

  lit_trans_scores = calculate_littransscore(bert_scores, bleurt_scores)
  test_df['Score'] = lit_trans_scores
  test_df['BLEURT'] = bleurt_scores
  test_df['BERT'] = bert_scores

  lit_trans_scores_scores_metaphor = test_df.groupby('Type').get_group('Metaphor')['Score']
  lit_trans_scores_scores_idiom = test_df.groupby('Type').get_group('Idiom')['Score']
  lit_trans_scores_scores_simile = test_df.groupby('Type').get_group('Simile')['Score']
  lit_trans_scores_scores_sarcasm = test_df.groupby('Type').get_group('Sarcasm')['Score']

  print(min(lit_trans_scores))
  print(max(lit_trans_scores))

  print('Average literal translation score for metaphor: %d' %(sum(lit_trans_scores_scores_metaphor)/len(lit_trans_scores_scores_metaphor)))
  print('Average literal translation score for idiom: %d' %(sum(lit_trans_scores_scores_idiom)/len(lit_trans_scores_scores_idiom)))
  print('Average literal translation score for simile: %d' %(sum(lit_trans_scores_scores_simile)/len(lit_trans_scores_scores_simile)))
  print('Average literal translation score for sarcasm: %d' %(sum(lit_trans_scores_scores_sarcasm)/len(lit_trans_scores_scores_sarcasm)))

In [ ]:
calculate_scores(df_fewshot_random)

[0.6963939070701599, 0.7511367797851562, 0.8317150473594666, 0.41311562061309814, 0.7311727404594421, 0.8332924246788025, 0.7621062397956848, 0.6821225881576538, 0.5626261830329895, 0.5217309594154358, 0.7752665877342224, 0.78742915391922, 0.661415159702301, 0.7924999594688416, 0.44037026166915894, 0.655007541179657, 0.7226647734642029, 0.6688984036445618, 0.6476841568946838, 0.6243807077407837, 0.7082386016845703, 0.8079298734664917, 0.7851011753082275, 0.6540867686271667, 0.6840981245040894, 0.9805935025215149, 0.6501232385635376, 0.6663578748703003, 0.7962833642959595, 0.5221862196922302, 0.841139018535614, 0.7633392214775085, 0.7312557101249695, 0.6577761173248291, 0.6934575438499451, 0.8315653204917908, 0.8677117824554443, 0.6219409108161926, 0.7666767239570618, 0.7184764742851257, 0.7431455850601196, 0.8187269568443298, 0.7098297476768494, 0.7666558623313904, 0.7713133692741394, 0.7976999878883362, 0.6636033654212952, 0.6321231126785278, 0.571516215801239, 0.8639445900917053, 0.7

<ipython-input-26-f48c1ffe6b80>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bertscore = load_metric('bertscore')


[0.9561523795127869, 0.9441882967948914, 0.9813277721405029, 0.876980721950531, 0.9601880311965942, 0.9667433500289917, 0.9504498243331909, 0.9061341881752014, 0.904304027557373, 0.9237983226776123, 0.9235543608665466, 0.9344609379768372, 0.8972330093383789, 0.9580388069152832, 0.8941972255706787, 0.916861891746521, 0.9164454340934753, 0.9226617813110352, 0.9164435267448425, 0.9192275404930115, 0.9428485035896301, 0.9616227149963379, 0.967494785785675, 0.93592768907547, 0.924767792224884, 0.9999998211860657, 0.9240302443504333, 0.9154191017150879, 0.9615767598152161, 0.9038680195808411, 0.9958382248878479, 0.9305011034011841, 0.9676269888877869, 0.9231008887290955, 0.9344848394393921, 0.9887343645095825, 0.9854015111923218, 0.936519205570221, 0.9326172471046448, 0.9441643953323364, 0.9662973284721375, 0.962681770324707, 0.9492109417915344, 0.9599388837814331, 0.9623932838439941, 0.9625689387321472, 0.9250217080116272, 0.9183041453361511, 0.925333559513092, 0.9708095192909241, 0.9777402

In [ ]:
def fewshot_knn(input, k, neighbours,df, type_filtering=True):
  few_shot_prompt = generate_few_shot_prompt(neighbours, df, type_filtering)
  task_des = ("For the given figurative sentences, provide a literal translation: \n\n") if not type_filtering else ""
  test_sample_prompt = ("Give literal translation for this " + get_type(input.Hypothesis) + "\n") if type_filtering else ""
  test_case = "Figurative sentence: " + input.Hypothesis + "\nLiteral translation: "
  prompt = task_des + few_shot_prompt +test_sample_prompt + test_case
  print(prompt, end = "")
  openai.api_key = aditya_api_key
  lit_output = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt, 
    max_tokens = 1500
  )
  return lit_output.choices[0].text.strip('\n')

In [ ]:
df_fewshot_random_type_filtering = test_df.copy()
df_fewshot_random_type_filtering["Literal Translation"] = [""]*df_fewshot_random_type_filtering.shape[0]
for i in range(0,df_fewshot_random_type_filtering.shape[0]):
  input = df_fewshot_random.iloc[i]
  pred = fewshot_random(input,3,False)
  print(pred)
  df_fewshot_random.loc[i,"Literal Translation"] = pred

In [ ]:
shuffled_train_df = train_df.dropna(subset = ["Premise"]).sample(frac=1)

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = np.array([model.encode(shuffled_train_df.iloc[0].Hypothesis, convert_to_numpy=True)])
print(embeddings.shape)
for i in range(1,shuffled_train_df.shape[0]):
  sample = shuffled_train_df.iloc[i].Hypothesis
  embeddings = np.append(embeddings,np.array([model.encode(sample, convert_to_numpy=True)]),axis=0)

(1, 384)


In [ ]:
import faiss
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [ ]:
def fewshot_knn_embeddings(test_df, type_filtering=False):
  for i in range(test_df.shape[0]):
    input = test_df.iloc[i]
    embed = np.array([model.encode(test_df.iloc[i].Hypothesis, convert_to_numpy=True)])
    D, I = index.search(embed, 3)
    neighbours = I[0]
    literal = fewshot_knn(input,3,neighbours,shuffled_train_df, type_filtering=type_filtering)
    test_df.loc[i,"Literal Translation"] = literal

In [ ]:
df_fewshot_knn_without_type = test_df.copy()
df_fewshot_knn_without_type["Literal Translation"] = [""]*df_fewshot_knn_without_type.shape[0]
fewshot_knn_embeddings(df_fewshot_knn_without_type,type_filtering=False)

For the given figurative sentences, provide a literal translation: 

Figurative sentence: I was so delighted when I heard someone bumping around the downstairs of my house last night after I had gone to bed.
Literal translation: I heard someone bumping around the downstairs of my house last night after I had gone to bed and it scared me half to death.

Figurative sentence: The sound of a knock at my door late at night made me feel so appreciated that someone cared about me at that hour.
Literal translation: I was terrified when I heard a knock at my door late at night

Figurative sentence: My alarm roared at me this morning.
Literal translation: My  alarm woke me up at the morning.

Figurative sentence: I heard a knock on my door at 2 am in the morning, and was super excited because I didn't know who it was.
Literal translation: For the given figurative sentences, provide a literal translation: 

Figurative sentence: I saw a person being horrible and racist to someone and it made me so

In [ ]:
df_fewshot_knn_without_type

,Unnamed: 0,Index,Hypothesis,Premise,Type,Explanation,Label,SarcasmSimilarity,Source,Literal Translation
0,331,50,I heard a knock on my door at 2 am in the morn...,I heard a knock on my door at 2 am in the morn...,Sarcasm,Unknown people knocking on someone's door at 2...,Contradiction,UNQ,FLUTE,I heard a knock on my door at 2 am in the mor...
1,233,1630,I was very proud of my sense of judgement when...,I was seriously doubting my sense of judgement...,Sarcasm,: Someone who is admired for their great visio...,Contradiction,UNQ,FLUTE,I was glad that I didn't trust that person wh...
2,66,2112,I adore it when people blast their music durin...,I despise it when people blast their music dur...,Sarcasm,People blasting their music during the wee hou...,Contradiction,UNQ,FLUTE,I hate it when people play loud music at 3am ...
3,374,1833,I am pleased because the last thing I need is ...,I've been getting really bad headaches recently,Sarcasm,Having a headache makes someone's day worse as...,Contradiction,UNQ,FLUTE,I am relieved because the last thing I need i...
4,452,1057,I was extremely happy to find out that all my ...,I was extremely broken to find out that all my...,Sarcasm,Studying for an annual exam is often a waste o...,Contradiction,UNQ,FLUTE,I was extremely disappointed to find out that...
...,...,...,...,...,...,...,...,...,...,...
395,332,3260,I know that woman too well to ever want to lay...,I know that woman too well to ever want to hur...,Idiom,To lay a finger on someone usually refers to h...,Entailment,UNQ,FLUTE,I know that woman too well to ever want to to...
396,527,7919,‘ That spoke volumes for the character as well...,‘ That provided substantial information for th...,Idiom,NaN,Entailment,NaN,IMPLI,‘That provided substantial information about ...
397,569,6303,It 's all water under the bridge ; you do n't ...,It 's all in the past ; you do n't want to hea...,Idiom,NaN,Entailment,NaN,IMPLI,It doesn't matter now; you don't want to be r...
398,222,6481,The road and railway from Mallaig go south han...,The road and railway from Mallaig go south in ...,Idiom,NaN,Entailment,NaN,IMPLI,The road and railway from Mallaig go south to...


In [ ]:
calculate_scores(df_fewshot_knn_without_type)

[0.6972810626029968, 0.6167876124382019, 0.7839049696922302, 0.4125124216079712, 0.8834017515182495, 0.8527848124504089, 0.7949019074440002, 0.6837688684463501, 0.524182140827179, 0.6627402901649475, 0.5324832797050476, 0.6447105407714844, 0.5450760126113892, 0.6905773282051086, 0.6802444458007812, 0.698113739490509, 0.9033690094947815, 0.7160366177558899, 0.6047033667564392, 0.6458204388618469, 0.8578654527664185, 0.6952971816062927, 0.7741562128067017, 0.6480491757392883, 0.7497105598449707, 0.7400150895118713, 0.6024995446205139, 0.48458194732666016, 0.8178381323814392, 0.4421672821044922, 0.6607961058616638, 0.6741771697998047, 0.73450767993927, 0.7232747673988342, 0.7475635409355164, 0.8315653204917908, 0.8193750977516174, 0.6290315985679626, 0.700030505657196, 0.6888786554336548, 0.8761833310127258, 0.8012823462486267, 0.6951020956039429, 0.7781174182891846, 0.6187069416046143, 0.7451139092445374, 0.6619600057601929, 0.7367211580276489, 0.6363251209259033, 0.8436897397041321, 0.6

In [ ]:
df_fewshot_knn_without_type.to_csv("Few-shot-KNN-sampled-from-full-train-without-type.csv")

In [ ]:
sarcasm = np.flatnonzero(shuffled_train_df["Type"]=="Sarcasm")
metaphor = np.flatnonzero(shuffled_train_df["Type"]=="Metaphor")
idiom = np.flatnonzero(shuffled_train_df["Type"]=="Idiom")
simile = np.flatnonzero(shuffled_train_df["Type"]=="Simile")

embeddings_sarc = np.array(list(map(lambda x:list(embeddings[x]), sarcasm)))
embeddings_meta = np.array(list(map(lambda x:list(embeddings[x]), metaphor)))
embeddings_idiom = np.array(list(map(lambda x:list(embeddings[x]), idiom)))
embeddings_simile = np.array(list(map(lambda x:list(embeddings[x]), simile)))

index_sarcasm = faiss.IndexFlatL2(embeddings_sarc.shape[1])
index_sarcasm.add(embeddings_sarc)

index_metaphor = faiss.IndexFlatL2(embeddings_meta.shape[1])
index_metaphor.add(embeddings_meta)

index_idiom = faiss.IndexFlatL2(embeddings_idiom.shape[1])
index_idiom.add(embeddings_idiom)

index_simile = faiss.IndexFlatL2(embeddings_simile.shape[1])
index_simile.add(embeddings_simile)

index_mapping = {"Sarcasm":index_sarcasm, "Metaphor":index_metaphor, "Idiom":index_idiom, "Simile":index_simile}

In [ ]:
def fewshot_knn_embeddings_with_type_filtering(test_df):
  for i in range(test_df.shape[0]):
    input = test_df.iloc[i]
    embed = np.array([model.encode(test_df.iloc[i].Hypothesis, convert_to_numpy=True)])
    type_index = index_mapping[input.Type]
    D, I = type_index.search(embed, 3)
    neighbours = I[0]
    literal = fewshot_knn(input,3,neighbours,shuffled_train_df)
    test_df.loc[i,"Literal Translation"] = literal

In [ ]:
df_fewshot_knn_with_type_filtering = test_df.copy()
df_fewshot_knn_with_type_filtering["Literal Translation"] = [""]*df_fewshot_knn_with_type_filtering.shape[0]
fewshot_knn_embeddings_with_type_filtering(df_fewshot_knn_with_type_filtering)

In [ ]:
#check for breaking
list(filter(lambda x:not df_fewshot_knn_with_type_filtering.iloc[x]["Literal Translation"],range(df_fewshot_knn_with_type_filtering.shape[0])))

[]

In [ ]:
calculate_scores(df_fewshot_knn_with_type_filtering)

[0.6592556238174438, 0.8131820559501648, 0.8053460717201233, 0.3747599124908447, 0.8084695935249329, 0.8467728495597839, 0.7146643400192261, 0.6899712085723877, 0.6746917366981506, 0.5765700936317444, 0.8700931072235107, 0.8169629573822021, 0.5981283187866211, 0.6975691914558411, 0.5775537490844727, 0.7090688943862915, 0.6429463028907776, 0.7425441145896912, 0.663982629776001, 0.6069027185440063, 0.7077821493148804, 0.6690120697021484, 0.7427046298980713, 0.778580904006958, 0.7527487277984619, 0.8183149695396423, 0.5862922668457031, 0.6480114459991455, 0.8184681534767151, 0.5230237245559692, 0.841139018535614, 0.7415334582328796, 0.642204999923706, 0.7612364292144775, 0.8294981122016907, 0.7313642501831055, 0.9348363280296326, 0.42530137300491333, 0.8060926795005798, 0.6708332896232605, 0.8248361945152283, 0.6298595666885376, 0.6453487873077393, 0.804869532585144, 0.727326512336731, 0.8370067477226257, 0.8168843388557434, 0.8036817908287048, 0.5780712366104126, 0.842515230178833, 0.714

In [ ]:
df_fewshot_knn_with_type_filtering.to_csv("Few-shot-KNN-with-type.csv")

In [ ]:
df_mapping = {"Sarcasm": sarcasm, "Metaphor": metaphor, "Simile": simile, "Idiom": idiom}

In [ ]:
import string
def get_pred(res, labels):
  text = res["choices"][0]["text"].translate(str.maketrans('', '', string.punctuation))
  pred_tokens = text.split(" ")
  pred = "Metaphor"
  for i in pred_tokens:
    if i in labels:
      pred = i
      break
  return pred.replace("'", "")

def get_type(text):
  openai.api_key = prathiksha_api_key
  res = openai.Completion.create(model='ada:ft-personal-2023-05-15-01-40-58', prompt=text, temperature=0, top_p=1.0, max_tokens = 100)
  return get_pred(res, ["Sarcasm","Simile","Metaphor","Idiom"])

In [ ]:
#testing which method is the best
test_indices = random.sample(range(test_df.shape[0]),5)
for i in test_indices:
  input = test_df.iloc[i]
  pred_type = get_type(input.Hypothesis)
  pred_random = fewshot_random(input,3,False)
  pred_knn_with_type = fewshot_knn(input,3,random.sample(list(df_mapping[pred_type]),3),shuffled_train_df)
  embed = np.array([model.encode(input.Hypothesis, convert_to_numpy=True)])
  D, I = index.search(embed, 3)
  neighbours = I[0]
  pred_knn_embeddings = fewshot_knn(input,3,neighbours,shuffled_train_df)
  type_index = index_mapping[input.Type]
  D1, I1 = type_index.search(embed, 3)
  neighbours1 = I1[0]
  pred_knn_embeddings_with_type = fewshot_knn(input,3,neighbours1,shuffled_train_df)

  print("Sample: ", input.Hypothesis)
  print("Actual: ", input.Premise)
  print("------------------------------------------------------------------------------------------------------------------------")
  print("Few shot Random: ", pred_random)
  print("Few shot random with type: ", pred_knn_with_type)
  print("Few shot knn: ", pred_knn_embeddings)
  print("Few shot knn with type: ", pred_knn_embeddings_with_type)
  print("\n\n")

KeyboardInterrupt: ignored